In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [4]:
# Preprocess images (resize and normalize)
def preprocess_images(images, labels):
    images = tf.image.resize(images, (224, 224)) / 255.0  # Resize to 224x224 and normalize
    return images, labels


In [5]:
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess_images).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [6]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess_images).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [7]:
# Load the VGG16 model without the top layer
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False  # Freeze the base layers initially

In [8]:
# Add a custom classification head
model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])


In [9]:
# Compile the model with initial hyperparameters
learning_rate = 0.001
model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [11]:
# Train the model
history = model.fit(train_ds, 
                    epochs=1, 
                    validation_data=test_ds, 
                    callbacks=[early_stopping, reduce_lr])

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2232s 1s/step - accuracy: 0.2449 - loss: 2.0856 - val_accuracy: 0.4502 - val_loss: 1.5522 - learning_rate: 0.0010


In [12]:
# Unfreeze some layers for fine-tuning
vgg_base.trainable = True  # Unfreeze the entire base model

In [13]:
# Recompile with a smaller learning rate for fine-tuning
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fine-tune the model
history_fine_tune = model.fit(train_ds, 
                              epochs=1, 
                              validation_data=test_ds, 
                              callbacks=[early_stopping, reduce_lr])

 340/1563 ━━━━━━━━━━━━━━━━━━━━ 5:45:36 17s/step - accuracy: 0.1141 - loss: 2.3375

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Final Test Accuracy: {test_acc:.2f}")